In [1]:
import nltk
nltk.download('brown')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('words')
nltk.download('wordnet')

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\parkc\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\parkc\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\parkc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\parkc\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\parkc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
from nltk.corpus import brown
from nltk.util import trigrams
from nltk.util import bigrams
from nltk.tokenize import word_tokenize
from nltk.collocations import *
import re
brown.categories()

['adventure',
 'belles_lettres',
 'editorial',
 'fiction',
 'government',
 'hobbies',
 'humor',
 'learned',
 'lore',
 'mystery',
 'news',
 'religion',
 'reviews',
 'romance',
 'science_fiction']

In [3]:
words_tags_list = list(map(list, zip(*nltk.pos_tag(brown.words(categories='news')))))

In [4]:
dict_tri_pos_freqdist = nltk.FreqDist(trigrams(words_tags_list[1]))
dict_tri_words_freqdist = nltk.FreqDist(trigrams(words_tags_list[0]))

In [6]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()
fourgram_measures = nltk.collocations.QuadgramAssocMeasures()

In [264]:
creature_filter = lambda *w: 'he am take' not in w
finder = BigramCollocationFinder.from_words(brown.words(categories='news'))
#finder.apply_freq_filter(1)
finder.apply_ngram_filter(creature_filter)
finder.nbest(bigram_measures.pmi, 100)

[]

In [43]:
token = 'running'
creature_filter = lambda *w: token not in w
finder = BigramCollocationFinder.from_words(brown.words())
finder.apply_freq_filter(10)
finder.apply_ngram_filter(creature_filter)
finder.nbest(trigram_measures.pmi, 10)

[(',', 'running')]

## 1. Generate candidate set

In [8]:
from nltk.metrics import jaccard_distance
from nltk.metrics.distance  import edit_distance
from nltk.corpus import words
from nltk.util import ngrams
from nltk.util import trigrams
from nltk.util import bigrams
from operator import itemgetter

In [9]:
def generate_pos_tags(tokens):
    return list(map(list, zip(*nltk.pos_tag(tokens))))

In [10]:
ind = {}
letters = 'abcdefghijklmnopqrstuvwxyz'
for word in words.words():
    #word_ = word.lower()
    if word[0].lower() not in ind:
        ind[word[0].lower()] = set()
    ind[word[0].lower()].add(word)
for letter in letters:
    if letter not in ind:
        ind[letter] = set()

In [6]:
brown_pos = generate_pos_tags(brown.words())
brown_bg = list(bigrams(brown_pos[0]))
brown_tg = list(trigrams(brown_pos[0]))
brown_bg_pos = list(bigrams(brown_pos[1]))
brown_tg_pos = list(trigrams(brown_pos[1]))
#Frequency distribution for Words
freq_dist = nltk.FreqDist(brown.words())
freq_dist_bigrams = nltk.FreqDist(list(bigrams(brown.words())))
freq_dist_trigrams = nltk.FreqDist(list(trigrams(brown.words())))
#Frequency distribution for POS tags
freq_dist_pos = nltk.FreqDist(brown_pos[1])
freq_dist_bg_pos = nltk.FreqDist(brown_bg_pos)
freq_dist_tg_pos = nltk.FreqDist(brown_tg_pos)

In [7]:
correct_spellings = words.words()
max_dist = 5
min_word = 10

## Non-word spelling correction

In [225]:
entries = word_tokenize(re.sub(r'[^\w]', ' ', 'He is runing to bus stp.'))
print(entries)
bigrams_ = list(bigrams(entries))
print(bigrams_)

['He', 'is', 'runing', 'to', 'bus', 'stp']
[('He', 'is'), ('is', 'runing'), ('runing', 'to'), ('to', 'bus'), ('bus', 'stp')]


In [227]:
for position, entry in enumerate(entries):
    entry = entry.lower()
    if entry not in correct_spellings:
        #%time temp_jaccard = [(jaccard_distance(set(ngrams(entry, 2)), set(ngrams(w, 2))),w) for w in correct_spellings if w[0]==entry[0]]
        %time temp_edit = [(edit_distance(entry, w),w) for w in correct_spellings if w[0]==entry[0]]
        print("Edit distance")
        i = 2
        %time edit_set = [x for x in temp_edit if x[0] <= i]
        while len(edit_set) < min_word & i < max_dist:
            i = i + 1
            edit_set = [x for x in temp_edit if x[0] <= i]
        freqs_1 = {}
        for word in edit_set:
            if word[1] in freq_dist:
                freqs_1[word[1]] = freq_dist[word[1]]
        entry_bigrams = [x for x in bigrams_ if (entry in x)]
        candidate_bigrams = {}
        #create bigrams candidate set
        for correction in edit_set:
            for pos_, x in enumerate(entry_bigrams):
                for j in range(len(x)):
                    if x[j] == entry:
                        y = list(x)
                        y[j] = correction[1]
                        x = tuple(y)
                if correction[1] not in candidate_bigrams:
                    candidate_bigrams[correction[1]] = list()
                candidate_bigrams[correction[1]].append(x)
        print(candidate_bigrams)
        freqs_2 = {}
        for candidate, cbigrams in candidate_bigrams.items():
            for cbigram in cbigrams: 
                if cbigram in freq_dist_bigrams:
                    if candidate not in freqs_2:
                        freqs_2[candidate] = 0
                    freqs_2[candidate] = freqs_2[candidate] + freq_dist_bigrams[cbigram]
        print('Frequency distribution by word')
        print(sorted(freqs_1.items(), key=itemgetter(1), reverse=True))
        print('Frequency distribution by bigrams')
        print(sorted(freqs_2.items(), key=itemgetter(1), reverse=True))
        print(position)

Wall time: 567 ms
Edit distance
Wall time: 1.03 ms
{'racing': [('is', 'racing'), ('racing', 'to')], 'raging': [('is', 'raging'), ('raging', 'to')], 'raking': [('is', 'raking'), ('raking', 'to')], 'randing': [('is', 'randing'), ('randing', 'to')], 'ranging': [('is', 'ranging'), ('ranging', 'to')], 'ranine': [('is', 'ranine'), ('ranine', 'to')], 'ranting': [('is', 'ranting'), ('ranting', 'to')], 'raping': [('is', 'raping'), ('raping', 'to')], 'rating': [('is', 'rating'), ('rating', 'to')], 'raving': [('is', 'raving'), ('raving', 'to')], 'reking': [('is', 'reking'), ('reking', 'to')], 'renin': [('is', 'renin'), ('renin', 'to')], 'rering': [('is', 'rering'), ('rering', 'to')], 'resing': [('is', 'resing'), ('resing', 'to')], 'riding': [('is', 'riding'), ('riding', 'to')], 'ring': [('is', 'ring'), ('ring', 'to'), ('is', 'ring'), ('ring', 'to')], 'ringing': [('is', 'ringing'), ('ringing', 'to')], 'rinsing': [('is', 'rinsing'), ('rinsing', 'to')], 'rising': [('is', 'rising'), ('rising', 'to')]

In [33]:
def get_candidates(entry, distance=2, min_word=10, max_dist=5):
    entry = entry.lower()
    temp_edit = [(edit_distance(entry, w),w) for w in ind[entry[0]]]
    i = distance
    edit_set = [x for x in temp_edit if x[0] <= i]
    while len(edit_set) < min_word & i < max_dist:
        i = i + 1
        edit_set = [x for x in temp_edit if x[0] <= i]
    edit_set = set([x[1] for x in edit_set])
    return edit_set

In [303]:
('is', 'taking') in brown_bg

True

In [309]:
('is', 'taking', 'to') in freq_dist_trigrams

FreqDist({("''", '?', '?'): 915, ("''", '.', '``'): 702, (',', 'and', 'the'): 652, ('.', 'It', 'is'): 529, ('.', 'It', 'was'): 518, ('.', 'In', 'the'): 398, ('?', '?', '``'): 381, ('.', '``', 'I'): 359, ("''", '!', '!'): 350, ("''", ',', 'he'): 346, ...})

In [ ]:
sentence = 'He is taken the basket to the laundry'
tokens = word_tokenize(sentence.lower())
raw = generate_pos_tags(tokens)
bgs = list(bigrams(raw[0]))
print(bgs)
bg_pos = list(bigrams(raw[1]))
for bg_entry in bgs:
    print(bg_entry)
    print(bg_entry in brown_bg)
    freqs3 = {}
    prob3 = {}
    if bg_entry not in brown_bg:
        for i, ent in enumerate(bg_entry):
            print(ent)
            candidates = get_candidates(ent, distance=5)
            for candidate in enumerate(candidates):
                temp = list(bg_entry)
                temp[i] = candidate[1]
                e = tuple(temp)
                if e in freq_dist_bigrams:
                    #print(e)
                    #print(freq_dist_bigrams[e])
                    freqs3[e] = freq_dist_bigrams[e]
                    prob3[e] = freq_dist_bigrams[e]/freq_dist[e[0]]
    print(sorted(prob3.items(), key=itemgetter(1), reverse=True))

[('he', 'is'), ('is', 'taken'), ('taken', 'the'), ('the', 'basket'), ('basket', 'to'), ('to', 'the'), ('the', 'laundry')]
('he', 'is')
True
[]
('is', 'taken')
True
[]
('taken', 'the')
True
[]
('the', 'basket')
True
[]
('basket', 'to')
False
basket
to
[(('bugle', 'to'), 1.0), (('bower', 'to'), 1.0), (('beau', 'to'), 1.0), (('backside', 'to'), 1.0), (('belong', 'to'), 0.6216216216216216), (('bonnet', 'to'), 0.5), (('butler', 'to'), 0.5), (('bonded', 'to'), 0.5), (('besieged', 'to'), 0.5), (('batter', 'to'), 0.5), (('begun', 'to'), 0.49019607843137253), (('bound', 'to'), 0.35714285714285715), (('boon', 'to'), 0.3333333333333333), (('bug', 'to'), 0.3333333333333333), (('bayonet', 'to'), 0.3333333333333333), (('begin', 'to'), 0.32926829268292684), (('blow', 'to'), 0.2727272727272727), (('bravery', 'to'), 0.25), (('blues', 'to'), 0.25), (('braces', 'to'), 0.25), (('beaten', 'to'), 0.23076923076923078), (('bother', 'to'), 0.22727272727272727), (('barrier', 'to'), 0.2222222222222222), (('blown

In [319]:
sentence = 'He is taken the basket to the laundry.'
tokens = word_tokenize(sentence)
raw = generate_pos_tags(tokens)
tg = raw[0]
tg_pos = list(trigrams(raw[1]))
print(tg)
print(tg_pos)
#print(tg_pos)
for tg_p in tg_pos:
    if tg_p not in freq_dist_tg_pos:
        pos_tg_correct = {}
        for pos_tg in freq_dist_tg_pos: 
            if pos_tg[0] == pos_tags[0]:
                pos_tg_correct[pos_tg] = freq_dist_tg_pos[pos_tg]/freq_dist_pos[pos_tags[0]]
        #sort by the frequency
        pos_tg_correct = sorted(pos_tg_correct.items(), key=itemgetter(1), reverse=True)
        print(pos_tg_correct)

['He', 'is', 'taken', 'the', 'basket', 'to', 'the', 'laundry', '.']
[('PRP', 'VBZ', 'VBN'), ('VBZ', 'VBN', 'DT'), ('VBN', 'DT', 'NN'), ('DT', 'NN', 'TO'), ('NN', 'TO', 'DT'), ('TO', 'DT', 'NN'), ('DT', 'NN', '.')]


In [347]:
len(nltk.FreqDist(brown_tg))

907494

In [55]:
t = ('is', 'walk', 'to')
tpos = list(trigrams(generate_pos_tags(t)[1]))[0]
pp = [brown_tg[i] for i, p in enumerate(brown_tg_pos) if p == tpos]
t_fd = {}
freqs_4 = {}
freqs_5 = {}
fd = nltk.FreqDist(pp)
for t_ in t:
    for p in fd:
        if t_ not in t_fd:
            t_fd[t_] = 0
        if t_ in p:
            #print(p)
            t_fd[t_] = t_fd[t_] + fd[p]
for i, q in enumerate(t_fd):
    if t_fd[q] == 0:
        candidates = get_candidates(q, distance=5)
        for candidate in candidates:
            r = list(t)
            r[i] = candidate
            r = tuple(r)
            rpos = list(trigrams(generate_pos_tags(r)[1]))
            #print(rpos)
            #print(freq_dist_tg_pos[rpos[0]])
            freqs_4[rpos[0]] = freq_dist_tg_pos[rpos[0]]
            freqs_5[candidate] = freq_dist_trigrams[candidate]
print(sorted(freqs_4.items(), key=itemgetter(1), reverse=True))
#print(sorted(freqs_4.items(), key=itemgetter(1), reverse=True))

[(('VBZ', 'VBN', 'TO'), 350), (('VBZ', 'JJ', 'TO'), 294), (('VBZ', 'RB', 'TO'), 104), (('VBZ', 'VBG', 'TO'), 76), (('VBZ', 'PRP', 'TO'), 58), (('VBZ', 'NN', 'TO'), 53), (('VBZ', 'IN', 'TO'), 21), (('VBZ', 'NNS', 'TO'), 11), (('VBZ', 'VBD', 'TO'), 11), (('VBZ', 'JJR', 'TO'), 10), (('VBZ', 'CD', 'TO'), 10), (('VBZ', 'NNP', 'TO'), 8), (('VBZ', 'WRB', 'TO'), 7), (('VBZ', 'DT', 'TO'), 5), (('VBZ', 'RBR', 'TO'), 5), (('VBZ', 'JJS', 'TO'), 1), (('VBZ', 'WP', 'TO'), 0), (('VBZ', 'MD', 'TO'), 0), (('VBZ', 'WP$', 'TO'), 0), (('VBZ', 'WDT', 'TO'), 0)]


In [338]:


print(list(nltk.FreqDist(pp)))
for fd in nltk.FreqDist(pp):
    for t_ in t:
        if t_ not in t_fd:
            t_fd[t_] = 0
        if t_ in fd:
            t_fd[t_] = t_fd[t_]
t_fd
#for i, tg_pos in enumerate(brown_tg_pos):
#    print(brown_tg[i])

[('has', 'been', 'the'), ('has', 'been', 'a'), ('has', 'become', 'a'), ('has', 'been', 'an'), ('has', 'been', 'no'), ('has', 'made', 'the'), ('has', 'made', 'a'), ('has', 'raised', 'the'), ('has', 'found', 'a'), ('has', 'seen', 'the'), ('has', 'followed', 'the'), ('is', 'called', 'the'), ('has', 'issued', 'an'), ('has', 'headed', 'the'), ('has', 'used', 'the'), ('has', 'been', 'some'), ('has', 'acquired', 'a'), ('has', 'taken', 'the'), ('has', 'led', 'the'), ('has', 'captured', 'the'), ('has', 'recognized', 'the'), ('has', 'studied', 'the'), ('has', 'had', 'no'), ('has', 'directed', 'a'), ('has', 'become', 'the'), ('has', 'achieved', 'a'), ('has', 'produced', 'a'), ('has', 'designed', 'a'), ('has', 'survived', 'the'), ('has', 'developed', 'the'), ('has', 'changed', 'the'), ('has', 'provided', 'a'), ('has', 'won', 'a'), ('has', 'preoccupied', 'the'), ('has', 'given', 'an'), ('has', 'handed', 'the'), ('is', 'followed', 'a'), ('has', 'caused', 'the'), ('has', 'aroused', 'the'), ('is', 'si

{'is': 0, 'take': 0, 'to': 0}

In [316]:
freq_dist_tg_pos

FreqDist({('IN', 'DT', 'NN'): 25135, ('DT', 'NN', 'IN'): 19413, ('DT', 'JJ', 'NN'): 19175, ('NN', 'IN', 'DT'): 17405, ('IN', 'DT', 'JJ'): 12668, ('JJ', 'NN', 'IN'): 10733, ('NN', 'IN', 'NN'): 7302, ('NNS', 'IN', 'DT'): 5652, ('DT', 'NN', 'NN'): 5594, ('IN', 'DT', 'NNP'): 5483, ...})

In [253]:
freq_dist_tg_pos[('VBP', 'VB', 'TO')]

61

In [257]:
freq_dist_tg_pos[('VBP', 'TO', 'VB')]

630

## Real word spelling correction

In [55]:
%time print(generate_pos_tags('I am running to bus stop.'))
%time print(generate_pos_tags('I am run to bus stop.'))

[['I', 'am', 'running', 'to', 'bus', 'stop', '.'], ['PRP', 'VBP', 'VBG', 'TO', 'VB', 'NN', '.']]
Wall time: 0 ns
[['I', 'am', 'run', 'to', 'bus', 'stop', '.'], ['PRP', 'VBP', 'VBN', 'TO', 'VB', 'NN', '.']]
Wall time: 3.98 ms


1161192

In [49]:
raw = generate_pos_tags((word_tokenize('I am run to bus store')))
wbgs = list(bigrams(raw[0]))
pbgs = list(bigrams(raw[1]))
wtgs = list(trigrams(raw[0]))
ptgs = list(trigrams(raw[1]))
for i, bg in enumerate(wbgs):
    print(bg)
    print(freq_dist_bigrams[bg])
    if freq_dist_bigrams[bg] == 0: #if bigrams never occurs        
        print(pbgs[i])
        print(freq_dist_bg_pos[pbgs[i]])
'''
for j, tg in enumerate(wtgs):
    print(tg)
    print(freq_dist_trigrams[tg])
    if freq_dist_trigrams[tg] == 0: #if trigrams never occurs
        print(ptgs[j])
        print(freq_dist_tg_pos[ptgs[j]])
'''

('I', 'am')
207
('am', 'run')
0
('VBP', 'VBN')
2287
('run', 'to')
6
('to', 'bus')
0
('TO', 'VB')
15376
('bus', 'store')
0
('VB', 'NN')
1635


'\nfor j, tg in enumerate(wtgs):\n    print(tg)\n    print(freq_dist_trigrams[tg])\n    if freq_dist_trigrams[tg] == 0: #if trigrams never occurs\n        print(ptgs[j])\n        print(freq_dist_tg_pos[ptgs[j]])\n'

In [50]:
'VBP' in freq_dist_bg_pos

False

In [32]:
freq_dist_tg_pos[('PRP', 'VBP', 'VBG')]

225

In [21]:
WORDS=brown.words(categories='news')
def known(words): return set(w for w in words if w in WORDS)

In [ ]:
for edit in list(edits1('run')):
    print(edit[0])
    print(edit in ind[edit[0]])

In [ ]:
correct_words = words.words()

word = 'rin'
incorrect_words = 'The quick brwon fxo jumps over rin the callobrtion dog'


#for word in incorrect_words:
    #temp = [(edit_distance(word, w),w) for w in correct_words if w[0]==word[0]]
    #print(temp)
#edits1_set_jac = sorted(temp, key = lambda val:val[0])[0]
for incorrect_word in word_tokenize(incorrect_words):
    if incorrect_word not in words.words():
        print("Wrong word: " + incorrect_word)
        freqs_1 = {}
        freqs_2 = {}
        %time edits1_set = set([edit for edit in list(edits1(incorrect_word)) if edit in ind[edit[0].lower()]])
        %time edits2_set = set([edit for edit in list(edits2(incorrect_word)) if edit in ind[edit[0].lower()]])
        for word_ in edits1_set:
            freqs_1[word_] = freq_dist[word_]
        for word_ in edits2_set:
            freqs_2[word_] = freq_dist[word_]
        print(freqs_1)
        print(freqs_2)
        #print(sorted(freqs_1, key = lambda val:val))

In [ ]:
words = "I rin everyday in the evening."
raw_pos_tags = generate_pos_tags(words)
bigrams_words = list(bigrams(raw_pos_tags[0]))
trigrams_words = list(trigrams(raw_pos_tags[0]))
bigrams_pos = list(bigrams(raw_pos_tags[1]))
trigrams_pos = list(trigrams(raw_pos_tags[1]))
print(bigrams_words)
print(trigrams_words)
print(bigrams_pos)
print(trigrams_pos)

In [94]:
generate_pos_tags("I am rain to the bus stop")

[['I', 'am', 'rain', 'to', 'the', 'bus', 'stop'],
 ['PRP', 'VBP', 'RB', 'TO', 'DT', 'NN', 'NN']]

In [133]:
generate_pos_tags("I am raining to the bus stop")

[['I', 'am', 'raining', 'to', 'the', 'bus', 'stop'],
 ['PRP', 'VBP', 'VBG', 'TO', 'DT', 'NN', 'NN']]

In [104]:
nltk.FreqDist(trigrams(words_tags_list[1]))[0]

0